In [9]:
import numpy as np
import pandas as pd
import sqlite3
import mgrs

In [15]:
conn = sqlite3.connect('C:/Users/balso/Downloads/FPA_FOD_20170508.sqlite')
ca_fires = pd.read_sql_query('select * from fires where State = "CA";', conn) # 

m = mgrs.MGRS() #mgrs API
to_mgrs_1m = [] #precision 5
to_mgrs_10m = [] #precision 4
to_mgrs_100m = [] #precision 3
to_mgrs_1km = []  #precision 2
to_mgrs_10km = []  #precision 1
to_mgrs_100km = []  #precision 0
to_mgrs_lat_10km = []
to_mgrs_lon_10km = []
ca_fires_data = pd.DataFrame()

ca_fires_data['object_id'] = ca_fires['OBJECTID']
ca_fires_data['reporting_unit'] = ca_fires['NWCG_REPORTING_UNIT_NAME']
ca_fires_data['fire_date_num'] = ca_fires['DISCOVERY_DATE']
ca_fires_data['fire_date_cal'] = pd.to_datetime(ca_fires['DISCOVERY_DATE'], unit='D', origin='julian') #convert numeric to calendar date
ca_fires_data['fire_year'] = ca_fires['FIRE_YEAR']
ca_fires_data['fire_month'] = ca_fires_data['fire_date_cal'].dt.month_name()
ca_fires_data['fire_day_of_week'] = ca_fires_data['fire_date_cal'].dt.day_name()
ca_fires_data['fire_class'] = ca_fires['FIRE_SIZE_CLASS']
ca_fires_data['lat'] = ca_fires['LATITUDE']
ca_fires_data['lon'] = ca_fires['LONGITUDE']

for i, j in zip(ca_fires_data['lat'].tolist(), ca_fires_data['lon'].tolist()):
    to_mgrs_1m.append(m.toMGRS(i, j, MGRSPrecision=5)) # use mgrs API to convert lat/lon to MGRS conventions
    to_mgrs_10m.append(m.toMGRS(i, j, MGRSPrecision=4)) 
    to_mgrs_100m.append(m.toMGRS(i, j, MGRSPrecision=3)) 
    to_mgrs_1km.append(m.toMGRS(i, j, MGRSPrecision=2)) 
    to_mgrs_10km.append(m.toMGRS(i, j, MGRSPrecision=1)) 
    to_mgrs_100km.append(m.toMGRS(i, j, MGRSPrecision=0)) 

ca_fires_data['mgrs_1m'] = to_mgrs_1m
ca_fires_data['mgrs_10m'] = to_mgrs_10m
ca_fires_data['mgrs_100m'] = to_mgrs_100m
ca_fires_data['mgrs_1km'] = to_mgrs_1km
ca_fires_data['mgrs_10km'] = to_mgrs_10km
ca_fires_data['mgrs_100km'] = to_mgrs_100km

for k in ca_fires_data['mgrs_10km'].tolist():
    to_mgrs_lat_10km.append(m.toLatLon(k)[0])
    to_mgrs_lon_10km.append(m.toLatLon(k)[1])

ca_fires_data['mgrs_lat_10km'] = to_mgrs_lat_10km
ca_fires_data['mgrs_lon_10km'] = to_mgrs_lon_10km

# filter out winter months where fires not likely
months_used = ['April', 'May', 'June', 'July', 'August', 'September', 'October','November'] 
ca_fires_data = ca_fires_data[ca_fires_data.fire_month.isin(months_used)]

# filter out class A fires, which are less than 1/4 acre
classes_used = ['B','C','D','E','F','G']
ca_fires_data = ca_fires_data[ca_fires_data.fire_class.isin(classes_used)]

# filter out fires before 2000 since weather data is sparse before year 2000
ca_fires_data = ca_fires_data[ca_fires_data.fire_year >= 2000]

# limit to only the MGRS locations in Southern CA
mgrs_100km = ['11SMT','11SNT','11SNS','11SMS','11SLT'] # redefine mgrs blocks
socal_fires_data = ca_fires_data[ca_fires_data.mgrs_100km.isin(mgrs_100km)]

# keep only necessary columns
socal_lite = pd.DataFrame()
socal_lite['mgrs_100km'] = socal_fires_data['mgrs_100km']
socal_lite['mgrs_10km'] = socal_fires_data['mgrs_10km']
# socal_lite['lat_mgrs'] = socal_fires_data['mgrs_lat_10km']
# socal_lite['lon_mgrs'] = socal_fires_data['mgrs_lon_10km']
socal_lite['lat_raw'] = ca_fires_data['lat']
socal_lite['lon_raw'] = ca_fires_data['lon']
socal_lite['date'] = socal_fires_data['fire_date_cal']
socal_lite = socal_lite.groupby(level=0).first()
# socal_lite.mgrs_10km.unique().shape
socal_lite.reset_index(drop=True, inplace = True)
socal_lite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8057 entries, 0 to 8056
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   mgrs_100km  8057 non-null   object        
 1   mgrs_10km   8057 non-null   object        
 2   lat_raw     8057 non-null   float64       
 3   lon_raw     8057 non-null   float64       
 4   date        8057 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 314.9+ KB


In [16]:
socal_lite.to_csv('fires_raw_coords.csv')

In [7]:
socal_fires_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189550 entries, 0 to 189549
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   fire_year   189550 non-null  int64         
 1   fire_date   189550 non-null  datetime64[ns]
 2   fire_class  189550 non-null  object        
 3   lat         189550 non-null  float64       
 4   lon         189550 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 7.2+ MB
